# Imports

In [9]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import os

root_dir_path = os.getcwd()
root_dir_path = os.path.abspath(os.path.split(root_dir_path)[0])

In [17]:
model_save_path = rf'{root_dir_path}/models/'

test_df = pd.read_csv(rf'{root_dir_path}/data/test_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)

test_data = TabularDataset(test_df)
label = "kpb_activity_code"

# Activity-Code Prediction

In [18]:
predictor = TabularPredictor.load(model_save_path)

predictor.model_best

'RandomForestEntr'

In [19]:

predicted_code = predictor.predict(data = test_data, model = predictor.model_best)
for i in predicted_code :   
    print(i)

test_df[label] = predicted_code

A_LSS_111_0
A_LSS_106_0
A_ÖPP_402_0
A_ÖPP_402_0
A_ÖPP_402_0
A_LSS_106_0
A_LSS_111_0
A_LSS_106_0
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ÖPP_405_0
A_ÖPP_405_0
A_ÖPP_405_0
A_ÖPP_405_0
A_ÖPP_405_0
A_ÖPP_405_0
A_ÖPP_405_0
A_ÖPP_405_0
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ÖPP_405_0
A_LSS_106_0
A_LSS_106_0
A_LSS_106_0
A_LSS_106_0
A_LSS_106_0
A_LSS_106_0
A_LSS_106_2
A_LSS_106_2
A_LSS_106_0
A_ADM_600_1
A_LSS_106_0
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ÖPP_405_0
A_ÖPP_405_0
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ÖPP_405_0
A_ADM_600_1
A_ÖPP_405_0
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_ÖPP_405_0
A_ADM_600_1
A_ADM_600_1
A_ADM_600_1
A_AD

In [20]:
test_df.head(5)

,clean_code,account_code,counterpart_code,business_code,activity_code,supplier,kpb_activity_code
0,100251,30102,EP0000,5290,NaN,NaN,A_LSS_111_0
1,100251,35100,ESF000,5290,NaN,NaN,A_LSS_106_0
2,100251,40230,EÖ0000,5290,NaN,TELECTROMONTAGE EL & TELETEKNIK AB,A_ÖPP_402_0
3,100251,40233,EÖ0000,5290,NaN,ADVANIA SVERIGE AB,A_ÖPP_402_0
4,100251,40234,EÖ0000,5290,NaN,TELE2 SVERIGE AB,A_ÖPP_402_0


In [21]:
test_df.to_csv(rf'{root_dir_path}/predicted_data/predicted_data.csv.csv', sep= '|')